# Digit Recognizer

Os arquivos de dados train.csv e test.csv contêm imagens em escala de cinza de dígitos desenhados à mão, de zero a nove.

Cada imagem tem 28 pixels de altura e 28 pixels de largura, totalizando 784 pixels no total. Cada pixel tem um único valor de pixel associado a ele, indicando a claridade ou a escuridão desse pixel, com números mais altos significando mais escuro. Esse valor de pixel é um inteiro entre 0 e 255, inclusive.

O conjunto de dados de treinamento, (train.csv), tem 785 colunas. A primeira coluna, chamada "label", é o dígito que foi desenhado pelo usuário. O restante das colunas contém os valores de pixel da imagem associada.

Cada coluna de pixel no conjunto de treinamento tem um nome como pixelx, onde x é um inteiro entre 0 e 783, inclusive. Para localizar esse pixel na imagem, suponha que tenhamos decomposto x como x = i * 28 + j, onde i e j são inteiros entre 0 e 27, inclusive. Em seguida, o pixelx está localizado na linha i e na coluna j de uma matriz 28 x 28 (indexação por zero).

Por exemplo, pixel31 indica o pixel que está na quarta coluna da esquerda e a segunda linha a partir do topo, como no diagrama ascii abaixo.

Visualmente, se omitirmos o prefixo "pixel", os pixels comporão a imagem da seguinte forma:

    000 001 002 003 ... 026 027
    028 029 030 031 ... 054 055
    056 057 058 059 ... 082 083
     |   |   |   |  ...  |   |
    728 729 730 731 ... 754 755
    756 757 758 759 ... 782 783 

O conjunto de dados de teste, (test.csv), é o mesmo que o conjunto de treinamento, exceto que ele não contém a coluna "label".

Ele sera utilizada para a medição da acuracia pelo kaggle. 

In [49]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [50]:
df_train = pd.read_csv('train.csv');
df_test = pd.read_csv('test.csv');

In [45]:
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Analise dos dados
Analizando os dados para tomar uma descisao de como processalos.

In [47]:
df_train.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [48]:
df_train.describe()

KeyboardInterrupt: 

In [ ]:
df_train['label'].count()

In [ ]:
df_train['label'].value_counts()

In [ ]:
# np.sort(df_train['label'].value_counts().unique())
# np.sort(df_train['label'].value_counts())

In [ ]:
df_train['label'].value_counts().plot(kind='bar');

In [ ]:
plt.scatter(['1','7','3','9','2','6','0','4','8','5'],df_train['label'].value_counts());

### Separando features e labels

In [ ]:
X = df_train.values[:,1:]
y = df_train.values[:,0]

In [ ]:
np.unique(y)

### obtendo valores para teste e treino

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42, stratify=y);

In [ ]:
from PIL import Image
import scipy.misc
import random

In [ ]:
x = df_train.values[:10,1:]

In [ ]:
def amostra_do_dataset(x_train, largura_img=12, altura_img=3, qnt_x=10, qnt_y=3):
    plt.figure(figsize=(largura_img, altura_img))
    x, y = qnt_x, qnt_y
    qnt_max = x*y
    for i in range(0, qnt_max):
        plt.axis("off")
        plt.subplot(y, x, i+1)
        image = Image.fromarray(x_train[random.randint(0, qnt_max)].reshape((28,28)))
        plt.imshow(image)
    plt.axis("off")    
    plt.show()
    plt.close()

In [ ]:
amostra_do_dataset(x)